<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_101/Misc/101-1-streamlit_llamacpp_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Install langchain, streamlit, and huggingface_hub modules for python

In [1]:
%pip install -q langchain huggingface_hub streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


* We will use [Mistral 7B Instruct v0.1 model](https://mistral.ai/) but since those models can be very large, we'll instead use a quantized (compressed) version from [Tom Jobins](https://huggingface.co/TheBloke) on [Hugging Face](https://huggingface.co/)
* The compressed version is still 4GB in size. So we'll pre-fetch it and cache it now so we can see the download progress. That way, when we call this again from Streamlit, we'll already have it on disk and it won't take nearly as long to start the streamlit app, as it does to download the model.

In [2]:
from huggingface_hub import hf_hub_download

(repo_id, model_file_name) = ("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", "mistral-7b-instruct-v0.1.Q4_0.gguf")
hf_hub_download(repo_id=repo_id, filename=model_file_name, repo_type="model")

mistral-7b-instruct-v0.1.Q4_0.gguf:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/45167a542b6fa64a14aea61a4c468bbbf9f258a8/mistral-7b-instruct-v0.1.Q4_0.gguf'

* We will install [LlamaCpp](https://python.langchain.com/docs/integrations/llms/llamacpp) for python but with CUDA support so we can leverage GPU.
* This model will also work without GPU support on standard CPU, but then it will be very slow to process our requests and respond back.
* This notebook has code that requires GPU so ensure you have the `T4 GPU` selected. Otherwise the program will crash without any errors on screen.

In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.13-cp310-cp310-manylinux_2_35_x86_64.whl size=4094742 sha256=e4c9a308138fd4e71406d3ec6175808493a5ff39f29ca0c9e2d4e6c7d2e8d7d8
  Stored in directory: /root/.cache/pip/wheels/d3/c9/89/ec02bbfa2283812eb24639ba52e929b9d773f86e4419a7da58
Successfully built llama-cpp-python


* The original code modified and tweaked to use LlamaCpp so we can use local llm instead of OpenAI.

In [9]:
%%writefile streaming_app.py
import streamlit as st
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
from huggingface_hub import hf_hub_download


# We don't want to reinitialize the llm with every user interaction so we ask Streamlit to cache it
# Downloading a model takes time so we only want to do it once and cache it.
# Once a model is downloaded, it has to be loaded, and the time it takes to load a model is directly
# proportional to its physical size. So we don't want to keep reloading the model. Hence, we ask streamlit
# to cache it instead.
@st.cache_resource
def download_model_and_prepare_llm():

    # We have already run these 2 lines earlier, so when Streamlit runs it again, since it already has the model
    # downloaded locally, it will do an early return with the model's path on disk

    (repo_id, model_file_name) = ("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", "mistral-7b-instruct-v0.1.Q4_0.gguf")

    model_path = hf_hub_download(repo_id=repo_id, filename=model_file_name, repo_type="model")

    # We will now load the model using the LlamaCpp interface which we will later pass to LangChain
    llama_llm = LlamaCpp(
            model_path=model_path,
            temperature=0,
            max_tokens=512,
            top_p=1,
            # callback_manager=callback_manager,
            n_gpu_layers=100,
            n_batch=512,
            n_ctx=4096,
            stop=["[INST]"],
            verbose=False,
            streaming=True,
            )

    return llama_llm


# We cache the llm chain until the system prompt changes. Then we re initialize it with the new prompt.
@st.cache_resource
def create_chain(system_prompt):

    # We get the model we pre-fetched earlier and loaded using LlamaCpp interface
    llama_llm = download_model_and_prepare_llm()

    # Mistral 7b is a foundational model. It won't work with chat prompt out of the box.
    # So we create a new prompt template based on their specifications.
    # We also pass in an initial "system prompt" to set the model's personality.
    template = """<s>[INST]{}[/INST]</s>

[INST]{}[/INST]""".format(system_prompt, "{question}")

    # We now prepare the prompt template using the template string above so we can create an llm chain using it
    prompt = PromptTemplate(template=template, input_variables=["question"])

    # We initialize the chain with the model and the prompt template.
    llm_chain = LLMChain(llm=llama_llm, prompt=prompt)

    return llm_chain


# We don't use stream handler at this time.
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)


# Create a sidebar text field to allow us to set and modify the initial prompt
with st.sidebar:
    system_prompt = st.text_area(label="Enter a system prompt to adjust the chatbot behavior",
                                 value="You are a helpful AI assistant who answers questions in short sentences.")

# We build our chain by passing the system prompt.
# It is worth noting that the chain will be rebuilt if we change the system prompt.
llm = create_chain(system_prompt)

# We initialize the chat history in streamlit session
# This is not the model's memory. Just a list of messages we passed to it and
# responses we received from it. This will be used to render the chat history.
if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How may I help you today?"}]

# We loop though the chat history in the session and display it in a chat like interface
for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

# When we receive user input from the chat input:
if prompt := st.chat_input():

    # 1. we add it to the chat history in the session memory and tag it as user message
    st.session_state.messages.append({"role": "user", "content": prompt})

    # 2. we add it to the chat history on the screen
    st.chat_message("user").write(prompt)

    # stream_handler = StreamHandler(st.empty())

    # 3. pass it to our llm chain as a prompt and get a response back
    response = llm.run(prompt)

    # 4. we add the response to the chat history in the session memory and tag it as AI response
    st.session_state.messages.append({"role":"assistant", "content": response})

    # 5. We append the response to the chat history on the screen
    st.chat_message("assistant").write(response)


Overwriting streaming_app.py


In [14]:
!streamlit run streaming_app.py &>/content/logs.txt &

## Find the IP of your instance

In [11]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

35.184.92.246
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [18]:
!npx localtunnel --port 8501

npx: installed 22 in 1.444s
your url is: https://many-banks-cover.loca.lt
^C
